# dataset is minist

In [1]:
import scipy.io as sio
import cupy as cp  #for gpu

In [2]:
datas=sio.loadmat('../datasets/mnist.mat')
trainX=cp.asarray(datas['Xtrain'])
trainy=cp.asarray(datas['ytrain'])
testX=cp.asarray(datas['Xtest'])
testy=cp.asarray(datas['ytest'])

In [3]:
print('trainX shape:',trainX.shape)
print('trainy shape:',trainy.shape)
print('testX shape:',testX.shape)
print('testy shape:',testy.shape)

trainX shape: (55000, 784)
trainy shape: (55000, 10)
testX shape: (10000, 784)
testy shape: (10000, 10)


# shinnosuke-gpu

In [4]:
from shinnosuke.models import Model
from shinnosuke.layers.Base import Input,Variable,Constant,Matmul,Add
from shinnosuke.layers.FC import Dense

In [5]:
X_input=Input(shape=(None,784))
X=Dense(500,activation='relu')(X_input)
X=Dense(10,activation='softmax')(X)
model=Model(inputs=X_input,outputs=X)
model.compile(optimizer='sgd',loss='sparse_categorical_crossentropy')
model.fit(trainX,trainy,batch_size=512,epochs=5,validation_ratio=0.)

 Epoch[1/5]
55296/55296 [==============================>] -8s -76ms/batch -batch_loss: 0.4600 -batch_acc: 0.8704 
 Epoch[2/5]
55296/55296 [==============================>] -7s -62ms/batch -batch_loss: 0.3461 -batch_acc: 0.9074 
 Epoch[3/5]
55296/55296 [==============================>] -8s -75ms/batch -batch_loss: 0.2463 -batch_acc: 0.9213 
 Epoch[4/5]
55296/55296 [==============================>] -8s -74ms/batch -batch_loss: 0.2294 -batch_acc: 0.9259 
 Epoch[5/5]
55296/55296 [==============================>] -10s -93ms/batch -batch_loss: 0.2010 -batch_acc: 0.9491 


In [6]:
acc,loss=model.evaluate(testX,testy)
print('test acc: %f, test loss: %f'%(acc,loss))

test acc: 0.937100, test loss: 0.225211


# Compare with keras-gpu

In [8]:
from keras.layers import Dense,Input
from keras.models import Model

In [9]:
trainX=cp.asnumpy(trainX)
trainy=cp.asnumpy(trainy)
testX=cp.asnumpy(testX)
testy=cp.asnumpy(testy)

there are only little differences to keras,such as shape need to specify batch_size with None,and for one-hot labels,use sparse_categorical_crossentropy instead of categorical_crossentropy

In [14]:
X_input=Input(shape=(784,))
X=Dense(500,activation='relu')(X_input)
X=Dense(10,activation='softmax')(X)
model=Model(inputs=X_input,outputs=X)
model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(trainX,trainy,batch_size=512,epochs=5)

Epoch 1/5
55000/55000 [==============================] - 2s 44us/step - loss: 1.8233 - acc: 0.5510: 0s - loss: 1.8394 - acc: 0.54
Epoch 2/5
55000/55000 [==============================] - 2s 34us/step - loss: 1.1809 - acc: 0.7771
Epoch 3/5
55000/55000 [==============================] - 2s 34us/step - loss: 0.8688 - acc: 0.8255
Epoch 4/5
55000/55000 [==============================] - 2s 34us/step - loss: 0.7111 - acc: 0.8462
Epoch 5/5
55000/55000 [==============================] - 2s 34us/step - loss: 0.6195 - acc: 0.8577


In [17]:
preds=model.evaluate(testX,testy)
print('test acc: %f, test loss: %f'%(preds[1],preds[0]))

10000/10000 [==============================] - 1s 143us/step
test acc: 0.873400, test loss: 0.559828


# shinnosuke is slower than keras,which will be optimized in the future,but shinnosuke has a better performance on both training set and test set